# Update data

## need to keep

In [ ]:
"""
insert into m_hk_filter_v3
with df0 as(
select distinct ifa from (select distinct ifa from m_hk
intersect
select distinct ifa from m_hk_filter_v3)
),
df1 as(
select ifa, timestamp, latitude, longitude, connection_type, ip_address 
from m_hk 
where date(timestamp) >= DATE('2022-03-06')
)
select b.*
from df0 a join df1 b 
on a.ifa = b.ifa
"""

## need to filter

In [ ]:
"""
create table m_hk_update_ifa_filter as
with df1 as(
select distinct ifa from m_hk
except
select distinct ifa from m_hk_filter_v3
)
select distinct ifa from df1
"""

In [ ]:
"""
create table m_hk_update_filter_v1 as
with df as(
select ifa, timestamp, latitude, longitude, connection_type, ip_address 
from m_hk 
where date(timestamp) >= DATE('2021-09-01')
),
df1 as(
select b.*
from m_hk_update_ifa_filter a join df b 
on a.ifa = b.ifa
),

df2 as(
select ifa, count(*) num_obs from df1
group by ifa
),

df3 as(
select * from df2
where num_obs >= 15
),

df4 as(
select b.* from df3 a join df1 b on a.ifa = b.ifa
),

df5 as (
select ifa, count(distinct date(timestamp)) nday from df4
group by ifa),

df6 as(
select * from df5
where nday>3
)

select b.* from df6 a join df4 b on a.ifa = b.ifa

"""

In [ ]:
"""
create table m_hk_update_filter_v2 as
with df1 as (
select ifa, count(distinct connection_type) as nct, count(distinct ip_address) as nip from m_hk_update_filter_v1
group by ifa
),
df2 as(
select * from df1 where nct > 1 and nip > 1
),
df3 as(
select b.* from df2 a join m_hk_update_filter_v1 b on a.ifa = b.ifa
),
--
df4 as(
select *, date(timestamp) day, ST_AsText(st_point(round(longitude,4),round(latitude,4))) point from df3
),
df5 as(
select ifa, count(distinct day) nday, count(distinct point) npoint from df4
group by ifa
),
df6 as(
select * from df5
where nday>15 and npoint<=1
),
df7 as(
select distinct ifa from df3  
except
select ifa from df6
)
select b.* from df7 a join df3 b on a.ifa = b.ifa

"""

In [ ]:
"""
insert into m_hk_filter_v3
select * from m_hk_update_filter_v2
"""

# home location

In [ ]:
# find home location
"""
create table m_hk_table2_weekday_v4 as
with df as(
select * from m_hk_filter_v3
where (day_of_week(timestamp) between 1 and 5) and (length(cast(latitude as varchar)) > 5) and (length(cast(longitude as varchar)) > 6)
),

df1 as (
SELECT *, ST_AsText(st_point(round(longitude,4),round(latitude,4))) location, date(distinct timestamp) date
FROM (select * from df where (hour(timestamp) between 0 and 8) or (hour(timestamp) between 20 and 23) )
),

df2 as (
select ifa, location, min(latitude) latitude, min(longitude) longitude, count(distinct date) ndays from df1
group by ifa, location
),

df3 as(
select *, ROW_NUMBER() OVER(PARTITION BY ifa ORDER BY ndays desc) rn from df2
),

df4 as(
select * from df3
where rn <=3
)

select a.*, b.name as hk_district, b.geometry hk_district_geometry from df4 a cross join m_hk_districts b
where ST_Contains(ST_Polygon(b.geometry), a.location)
"""

In [ ]:
# find buildings around home location
"""
create table m_hk_table2_weekday_v4_1 as
with a as(
select * from m_hk_table2_weekday_v4
where (hk_district = 'Central and Western') 
),

b as(
select * from m_hk_district_building
where district = 'Central and Western'
)

select a.*, b.building_id, b.centroid_lat building_lat,b.centroid_lon building_lon, 
b.name building_name, b.tag building_tag,
great_circle_distance(a.latitude, a.longitude, b.centroid_lat,b.centroid_lon) dist 
from a 
inner join 
b 
on great_circle_distance(a.latitude, a.longitude, b.centroid_lat,b.centroid_lon)<=0.3
"""

In [ ]:
# find closest buildings for home location
"""
create table m_hk_table2_weekday_v4_2 as
with df1 as(
select *, ROW_NUMBER() OVER (PARTITION BY ifa, rn ORDER BY dist ASC) dist_rn from m_hk_table2_weekday_v3_1
),
df2 as(
select * from df1 where dist_rn = 1
),

df3 as(
select * from df2
where (building_tag = 'dormitory') or (building_tag = 'residential') or (building_tag = 'hotel') or 
(building_tag = 'apartments') or (building_tag = 'house') or (building_tag = 'houseboat') or  
(building_tag = 'nursing_home')  or 
(building_tag = '觀瀾雅軒 Baycrest') or (building_tag = 'building') or (building_tag = 'building]') 
or (building_tag = 'stilt_house') or (building_tag = 'shed') or (building_tag = 'pavilion') or 
(building_tag = 'podium') or (building_tag = 'terrace') or (building_tag = 'TS') or 
(building_tag = 'detached') or (building_tag = 'bungalow') or (building_tag = 'hut')
),
df4 as(
select *, ROW_NUMBER() OVER(PARTITION BY ifa ORDER BY rn ASC) home_rn from df3
)

select * from df4 where home_rn = 1

"""

# mobility data merge with home building

In [ ]:

"""
create table m_hk_filter_w_home_v4_4 as 
with df1 as(
select a.ifa, a.timestamp,a.latitude, a.longitude, b.name as hk_district 
from m_hk_filter_v3 a cross join m_hk_districts b
where ST_Contains(ST_Polygon(b.geometry), st_point(a.longitude,a.latitude))
),
df2 as(
select *, 
LAG(latitude) OVER (PARTITION BY ifa, date(timestamp) ORDER BY timestamp) as previous_latitude,
LAG(longitude) OVER (PARTITION BY ifa, date(timestamp) ORDER BY timestamp) as previous_longitude,
LAG(timestamp) OVER (PARTITION BY ifa, date(timestamp) ORDER BY timestamp) as previous_timestamp
from df1),
df3 as(
select * from df2
where great_circle_distance(previous_latitude,previous_longitude,latitude,longitude) >= 0.1
)
select b.*, a.latitude home_latitude, a.longitude home_longitude,a.hk_district home_district, 
a.building_id, a.building_lat, a.building_lon, a.building_tag 
from m_hk_table2_weekday_v4_2 a 
left join df3 b 
on a.ifa = b.ifa
"""

# building level flowmap

# stay at home

In [ ]:
"""
create table m_hk_Shun_Tin_stayhomehour as
with df1 as(
select * from m_hk_filter_w_home_v4_4 
where st_contains('POLYGON ((114.2241789 22.3268612, 114.2242047 22.3264248, 114.22467 22.3259585, 114.2250837 22.3254682, 114.2254197 22.3252769, 114.2258463 22.3252112, 114.226066 22.3250139, 114.22615 22.3247807, 114.2263892 22.3245655, 114.2263569 22.3244639, 114.2265443 22.3243144, 114.2268933 22.3242785, 114.227113 22.3242965, 114.2272487 22.3241769, 114.2276088 22.3241892, 114.2279026 22.324436, 114.227847 22.3246294, 114.2281568 22.3249331, 114.2281197 22.3249649, 114.228305 22.325129, 114.2284347 22.3251657, 114.2288965 22.3251477, 114.2289218 22.3254228, 114.2287947 22.3255526, 114.2286783 22.3255698, 114.22853 22.3257608, 114.2274473 22.3262138, 114.2271058 22.3260693, 114.2267379 22.3261575, 114.2265367 22.3266227, 114.2252359 22.3271685, 114.2248663 22.3272995, 114.2247397 22.3273452, 114.2245602 22.3273593, 114.2244142 22.327296, 114.2242693 22.3271661, 114.2241789 22.3268612))', st_point(building_lon,building_lat))
),

df2 as(
select *, st_contains('POLYGON ((114.2241789 22.3268612, 114.2242047 22.3264248, 114.22467 22.3259585, 114.2250837 22.3254682, 114.2254197 22.3252769, 114.2258463 22.3252112, 114.226066 22.3250139, 114.22615 22.3247807, 114.2263892 22.3245655, 114.2263569 22.3244639, 114.2265443 22.3243144, 114.2268933 22.3242785, 114.227113 22.3242965, 114.2272487 22.3241769, 114.2276088 22.3241892, 114.2279026 22.324436, 114.227847 22.3246294, 114.2281568 22.3249331, 114.2281197 22.3249649, 114.228305 22.325129, 114.2284347 22.3251657, 114.2288965 22.3251477, 114.2289218 22.3254228, 114.2287947 22.3255526, 114.2286783 22.3255698, 114.22853 22.3257608, 114.2274473 22.3262138, 114.2271058 22.3260693, 114.2267379 22.3261575, 114.2265367 22.3266227, 114.2252359 22.3271685, 114.2248663 22.3272995, 114.2247397 22.3273452, 114.2245602 22.3273593, 114.2244142 22.327296, 114.2242693 22.3271661, 114.2241789 22.3268612))', st_point(longitude,latitude)) as at_home, st_contains('POLYGON ((114.2241789 22.3268612, 114.2242047 22.3264248, 114.22467 22.3259585, 114.2250837 22.3254682, 114.2254197 22.3252769, 114.2258463 22.3252112, 114.226066 22.3250139, 114.22615 22.3247807, 114.2263892 22.3245655, 114.2263569 22.3244639, 114.2265443 22.3243144, 114.2268933 22.3242785, 114.227113 22.3242965, 114.2272487 22.3241769, 114.2276088 22.3241892, 114.2279026 22.324436, 114.227847 22.3246294, 114.2281568 22.3249331, 114.2281197 22.3249649, 114.228305 22.325129, 114.2284347 22.3251657, 114.2288965 22.3251477, 114.2289218 22.3254228, 114.2287947 22.3255526, 114.2286783 22.3255698, 114.22853 22.3257608, 114.2274473 22.3262138, 114.2271058 22.3260693, 114.2267379 22.3261575, 114.2265367 22.3266227, 114.2252359 22.3271685, 114.2248663 22.3272995, 114.2247397 22.3273452, 114.2245602 22.3273593, 114.2244142 22.327296, 114.2242693 22.3271661, 114.2241789 22.3268612))', st_point(previous_longitude,previous_latitude)) as previous_at_home from df1
)

select ifa, date(timestamp) date,building_id, building_lat, building_lon, 24-sum(date_diff('second',previous_timestamp, timestamp))/60.0/60.0 as time_st_home from df2 
where (at_home = false) and (previous_at_home = false)
group by ifa, date(timestamp),building_id, building_lat, building_lon
"""

In [ ]:
"""
create table m_hk_Lei_Tung_stayhomehour as
with df1 as(
select * from m_hk_filter_w_home_v4_4 
where st_contains('POLYGON ((114.1527177 22.2412778, 114.1528305 22.2410098, 114.1528846 22.2408858, 114.152911 22.2408235, 114.1528952 22.2407196, 114.1529253 22.2407047, 114.1529274 22.2405607, 114.152957 22.2404508, 114.1530377 22.2403993, 114.153104 22.2402951, 114.1531983 22.2402627, 114.1533806 22.2402776, 114.1535205 22.2403218, 114.1535659 22.2402723, 114.1535689 22.2402354, 114.153625 22.2400912, 114.1537992 22.2402002, 114.1539123 22.2402002, 114.1539513 22.2402157, 114.1539652 22.2402752, 114.1542705 22.2402801, 114.15451 22.2403593, 114.1547164 22.240424, 114.1547781 22.240441, 114.154928 22.2404611, 114.155347 22.2403513, 114.1561446 22.2405157, 114.1563529 22.2405607, 114.1564624 22.2404514, 114.1565366 22.240298, 114.1567883 22.2401582, 114.1571085 22.2401408, 114.1571764 22.2402302, 114.1572489 22.2402325, 114.1573204 22.2403891, 114.1574336 22.240394, 114.1574553 22.2403344, 114.1574929 22.2403073, 114.1575805 22.2403095, 114.1576483 22.2402225, 114.1577556 22.2401779, 114.1578941 22.2401807, 114.1580075 22.2402524, 114.1580933 22.240337, 114.1581095 22.2403148, 114.1582915 22.2404461, 114.1585062 22.2406591, 114.1585013 22.2407172, 114.1584557 22.2406863, 114.1583916 22.240741, 114.1584657 22.240819, 114.1585304 22.2410422, 114.1585195 22.2411738, 114.1586936 22.2414216, 114.1587016 22.2415238, 114.1586693 22.2416726, 114.1586033 22.241873, 114.1586319 22.2420476, 114.1587398 22.2423829, 114.1586774 22.2424264, 114.158555 22.242589, 114.1586645 22.2427551, 114.1586886 22.2428123, 114.1586784 22.2428695, 114.1584448 22.2430329, 114.1581759 22.2432065, 114.1579993 22.2432516, 114.1579566 22.2432443, 114.1579696 22.2425486, 114.157991 22.2423925, 114.1580126 22.2423251, 114.157924 22.2423258, 114.1578495 22.2423283, 114.15764 22.2423327, 114.157578 22.2423521, 114.1573852 22.2420577, 114.1570392 22.2420647, 114.1568243 22.2423997, 114.1563073 22.2423947, 114.1560595 22.2426857, 114.1559523 22.2426859, 114.155535 22.2427098, 114.1556008 22.2426659, 114.155526 22.2425766, 114.1554189 22.2425119, 114.1552312 22.2424569, 114.1549312 22.2424004, 114.1548455 22.242412, 114.1545312 22.2422512, 114.1545154 22.2422305, 114.1545154 22.2421421, 114.1544935 22.2421244, 114.1545156 22.2420742, 114.1544427 22.2420395, 114.1543949 22.2420001, 114.1543273 22.2419285, 114.1541934 22.2419958, 114.1541639 22.2419658, 114.1540114 22.2419579, 114.1537643 22.2420249, 114.1535924 22.2419753, 114.1536875 22.2418738, 114.1536735 22.2418189, 114.1534877 22.2416009, 114.1534398 22.2416679, 114.1533644 22.2417148, 114.1533238 22.2417147, 114.1532066 22.2417669, 114.1531586 22.2417247, 114.1530474 22.2416993, 114.1529921 22.2416527, 114.1529224 22.2416353, 114.1527993 22.2415111, 114.1527204 22.2414045, 114.1527177 22.2412778))', st_point(building_lon,building_lat))
),

df2 as(
select *, st_contains('POLYGON ((114.1527177 22.2412778, 114.1528305 22.2410098, 114.1528846 22.2408858, 114.152911 22.2408235, 114.1528952 22.2407196, 114.1529253 22.2407047, 114.1529274 22.2405607, 114.152957 22.2404508, 114.1530377 22.2403993, 114.153104 22.2402951, 114.1531983 22.2402627, 114.1533806 22.2402776, 114.1535205 22.2403218, 114.1535659 22.2402723, 114.1535689 22.2402354, 114.153625 22.2400912, 114.1537992 22.2402002, 114.1539123 22.2402002, 114.1539513 22.2402157, 114.1539652 22.2402752, 114.1542705 22.2402801, 114.15451 22.2403593, 114.1547164 22.240424, 114.1547781 22.240441, 114.154928 22.2404611, 114.155347 22.2403513, 114.1561446 22.2405157, 114.1563529 22.2405607, 114.1564624 22.2404514, 114.1565366 22.240298, 114.1567883 22.2401582, 114.1571085 22.2401408, 114.1571764 22.2402302, 114.1572489 22.2402325, 114.1573204 22.2403891, 114.1574336 22.240394, 114.1574553 22.2403344, 114.1574929 22.2403073, 114.1575805 22.2403095, 114.1576483 22.2402225, 114.1577556 22.2401779, 114.1578941 22.2401807, 114.1580075 22.2402524, 114.1580933 22.240337, 114.1581095 22.2403148, 114.1582915 22.2404461, 114.1585062 22.2406591, 114.1585013 22.2407172, 114.1584557 22.2406863, 114.1583916 22.240741, 114.1584657 22.240819, 114.1585304 22.2410422, 114.1585195 22.2411738, 114.1586936 22.2414216, 114.1587016 22.2415238, 114.1586693 22.2416726, 114.1586033 22.241873, 114.1586319 22.2420476, 114.1587398 22.2423829, 114.1586774 22.2424264, 114.158555 22.242589, 114.1586645 22.2427551, 114.1586886 22.2428123, 114.1586784 22.2428695, 114.1584448 22.2430329, 114.1581759 22.2432065, 114.1579993 22.2432516, 114.1579566 22.2432443, 114.1579696 22.2425486, 114.157991 22.2423925, 114.1580126 22.2423251, 114.157924 22.2423258, 114.1578495 22.2423283, 114.15764 22.2423327, 114.157578 22.2423521, 114.1573852 22.2420577, 114.1570392 22.2420647, 114.1568243 22.2423997, 114.1563073 22.2423947, 114.1560595 22.2426857, 114.1559523 22.2426859, 114.155535 22.2427098, 114.1556008 22.2426659, 114.155526 22.2425766, 114.1554189 22.2425119, 114.1552312 22.2424569, 114.1549312 22.2424004, 114.1548455 22.242412, 114.1545312 22.2422512, 114.1545154 22.2422305, 114.1545154 22.2421421, 114.1544935 22.2421244, 114.1545156 22.2420742, 114.1544427 22.2420395, 114.1543949 22.2420001, 114.1543273 22.2419285, 114.1541934 22.2419958, 114.1541639 22.2419658, 114.1540114 22.2419579, 114.1537643 22.2420249, 114.1535924 22.2419753, 114.1536875 22.2418738, 114.1536735 22.2418189, 114.1534877 22.2416009, 114.1534398 22.2416679, 114.1533644 22.2417148, 114.1533238 22.2417147, 114.1532066 22.2417669, 114.1531586 22.2417247, 114.1530474 22.2416993, 114.1529921 22.2416527, 114.1529224 22.2416353, 114.1527993 22.2415111, 114.1527204 22.2414045, 114.1527177 22.2412778))', st_point(longitude,latitude)) as at_home, st_contains('POLYGON ((114.1527177 22.2412778, 114.1528305 22.2410098, 114.1528846 22.2408858, 114.152911 22.2408235, 114.1528952 22.2407196, 114.1529253 22.2407047, 114.1529274 22.2405607, 114.152957 22.2404508, 114.1530377 22.2403993, 114.153104 22.2402951, 114.1531983 22.2402627, 114.1533806 22.2402776, 114.1535205 22.2403218, 114.1535659 22.2402723, 114.1535689 22.2402354, 114.153625 22.2400912, 114.1537992 22.2402002, 114.1539123 22.2402002, 114.1539513 22.2402157, 114.1539652 22.2402752, 114.1542705 22.2402801, 114.15451 22.2403593, 114.1547164 22.240424, 114.1547781 22.240441, 114.154928 22.2404611, 114.155347 22.2403513, 114.1561446 22.2405157, 114.1563529 22.2405607, 114.1564624 22.2404514, 114.1565366 22.240298, 114.1567883 22.2401582, 114.1571085 22.2401408, 114.1571764 22.2402302, 114.1572489 22.2402325, 114.1573204 22.2403891, 114.1574336 22.240394, 114.1574553 22.2403344, 114.1574929 22.2403073, 114.1575805 22.2403095, 114.1576483 22.2402225, 114.1577556 22.2401779, 114.1578941 22.2401807, 114.1580075 22.2402524, 114.1580933 22.240337, 114.1581095 22.2403148, 114.1582915 22.2404461, 114.1585062 22.2406591, 114.1585013 22.2407172, 114.1584557 22.2406863, 114.1583916 22.240741, 114.1584657 22.240819, 114.1585304 22.2410422, 114.1585195 22.2411738, 114.1586936 22.2414216, 114.1587016 22.2415238, 114.1586693 22.2416726, 114.1586033 22.241873, 114.1586319 22.2420476, 114.1587398 22.2423829, 114.1586774 22.2424264, 114.158555 22.242589, 114.1586645 22.2427551, 114.1586886 22.2428123, 114.1586784 22.2428695, 114.1584448 22.2430329, 114.1581759 22.2432065, 114.1579993 22.2432516, 114.1579566 22.2432443, 114.1579696 22.2425486, 114.157991 22.2423925, 114.1580126 22.2423251, 114.157924 22.2423258, 114.1578495 22.2423283, 114.15764 22.2423327, 114.157578 22.2423521, 114.1573852 22.2420577, 114.1570392 22.2420647, 114.1568243 22.2423997, 114.1563073 22.2423947, 114.1560595 22.2426857, 114.1559523 22.2426859, 114.155535 22.2427098, 114.1556008 22.2426659, 114.155526 22.2425766, 114.1554189 22.2425119, 114.1552312 22.2424569, 114.1549312 22.2424004, 114.1548455 22.242412, 114.1545312 22.2422512, 114.1545154 22.2422305, 114.1545154 22.2421421, 114.1544935 22.2421244, 114.1545156 22.2420742, 114.1544427 22.2420395, 114.1543949 22.2420001, 114.1543273 22.2419285, 114.1541934 22.2419958, 114.1541639 22.2419658, 114.1540114 22.2419579, 114.1537643 22.2420249, 114.1535924 22.2419753, 114.1536875 22.2418738, 114.1536735 22.2418189, 114.1534877 22.2416009, 114.1534398 22.2416679, 114.1533644 22.2417148, 114.1533238 22.2417147, 114.1532066 22.2417669, 114.1531586 22.2417247, 114.1530474 22.2416993, 114.1529921 22.2416527, 114.1529224 22.2416353, 114.1527993 22.2415111, 114.1527204 22.2414045, 114.1527177 22.2412778))', st_point(previous_longitude,previous_latitude)) as previous_at_home from df1
)

select ifa, date(timestamp) date,building_id, building_lat, building_lon, 24-sum(date_diff('second',previous_timestamp, timestamp))/60.0/60.0 as time_st_home from df2 
where (at_home = false) and (previous_at_home = false)
group by ifa, date(timestamp),building_id, building_lat, building_lon
"""